# Lab 10: Creating a Feature Matrix using Spark-ML Pipelines

As always, create a SparkContext/HiveContext.

In [ ]:
# Set up Spark Context
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import *

SparkContext.setSystemProperty('spark.executor.memory', '2g')
conf = SparkConf()
conf.set('spark.executor.instances', 15)
conf.set('spark.sql.autoBroadcastJoinThreshold', 200*1024*1024)  # 200MB for broadcast join
sc = SparkContext('yarn-client', 'Spark-lab10', conf=conf)

from pyspark.sql import HiveContext
hc = HiveContext(sc)
hc.sql("use demo")

Load the crimes_wn table that was created in Lab 9 as a Spark Dataframe, and filter the dataset to include all records between 2011 and 2014:

In [ ]:
crimes_wn_all = <YOUR CODE HERE>
crimes_wn = crimes_wn_all.filter(<YOUR CODE HERE>)

Creater a new data frame 'crimes' via feature transformations as follows:
- Create features for 'year', 'month' and 'day' from the raw field 'date'
- Create the 'hour' feature from the raw field 'time'
- Create a feature 'resolved' from the 'resolution' field with a value of 0.0 if "NONE" and 1.0 otherwise
- Include these other fields: category, district, dayofweek, description, neighborhood

In [ ]:
crimes = crimes_wn.<YOUR CODE HERE>
crimes.cache()
crimes.limit(5).toPandas()

Now create the weather dataframe that includes the precipitation, tmin and tmax for San Francisco for each day in the years 2011-2014

In [ ]:
weather = hc.<YOUR CODE HERE>
prcp = weather.<YOUR CODE HERE>
tmin = weather.<YOUR CODE HERE>
tmax = weather.<YOUR CODE HERE>

wdata = <YOUR CODE HERE>
wdata.limit(5).toPandas()

Join the resulting weather dataset with the crimes dataset, using the join key (year,month,day). 
* create a dataframe "joined" that joins the two dataframes (crimes and weather)
* In the joined dataset the year, month and day fields appear twice. Create a python list "keep" that includes the names of the fields that we would like to keep from the joined result, then create the "fm" dataframe by selecting those fields only.

In [ ]:
joined = <YOUR CODE HERE>
keep = <YOUR CODE HERE>
fm = joined.select(*keep).cache()

fm.limit(5).toPandas()

Store this feature matrix into HDFS, so that you can use it in the next lab. This is not strictly necessary when using Spark since a DataFrame can be used sequentially for the next step, but useful in our case since our application is split into individual labs.

Use the DataFrames save() function with the ORC data source. 

In [ ]:
fm.write.format("orc").save("/tmp/fm", mode="overwrite")